In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import pymc as pm
import logging
import time

import scipy as sp
import aesara.tensor as at 
import aesara
# Enable on-the-fly graph computations, but ignore
# absence of intermediate test values.
#theano.config.compute_test_value = "ignore"

# Set up logging.
logger = logging.getLogger()
logger.setLevel(logging.INFO)
from matplotlib import pyplot as plt

plt.style.use("seaborn-darkgrid")
print(f"Running on PyMC v{pm.__version__}")

Running on PyMC v4.3.0


/var/folders/54/_py3f4ks6f9d0x8d0wfy1pkc0000gn/T/ipykernel_7600/970271437.py:20: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-darkgrid")


In [2]:
class PMF_vanilla:
    def __init__(self, train, dim):
        """Build the Probabilistic Matrix Factorization model using pymc3.



        """
        self.dim = dim
    
      
        self.data = train.copy()
        self.n, self.m = self.data.shape
        self.map = None
        self.bounds = (0,1)


        # Perform mean value imputation
    
        
        # Low precision reflects uncertainty; prevents overfitting.
        # Set to the mean variance across users and items.
        self.alpha_u = (np.mean(self.data, axis=1).mean())**2 / np.std(self.data, axis=1).mean()
        self.alpha_v = (np.mean(self.data, axis=0).mean())**2 / np.std(self.data, axis=0).mean()

        self.beta_u = (np.mean(self.data, axis=1).mean()) / np.std(self.data, axis=1).mean()
        self.beta_v = (np.mean(self.data, axis=0).mean()) / np.std(self.data, axis=0).mean()
       
        self.bias = self.data.mean()
        #specify model coords
        coords = {
        "drug": np.arange(self.n),
        "latent": np.arange(self.dim),
        "side_effect": np.arange(self.m),
            }
        # Specify the model.
        logging.info("building the PMF model")

        with pm.Model(coords=coords) as pmf:
            U = pm.Gamma(
                "U",
                alpha=self.alpha_u,
                beta=self.beta_u,
                dims=("drug", "latent"),
                #testval=np.random.randn(n, dim) * std,
            )
      
            V = pm.Gamma(
                "V",
                alpha=self.alpha_v,
                beta=self.beta_v,
                dims=("side_effect", "latent"),
                #testval=np.random.randn(m, dim) * std,
            )
    

            R = pm.Poisson(
                "R", mu=(U @ V.T), observed=self.data , dims = ("drug", "side_effect")
            )

        logging.info("done building the PMF model")
        self.model = pmf

    def __str__(self):
        return self.name
    
    def find_map(self):
    #"""Find mode of posterior using L-BFGS-B optimization."""
        tstart = time.time()
        with self.model:
            logging.info("finding PMF MAP using L-BFGS-B optimization...")
            self.map = pm.find_MAP(method="L-BFGS-B")

        elapsed = int(time.time() - tstart)
        logging.info("found PMF MAP in %d seconds" % elapsed)
        return self.map


    def map(self):
        try:
            return self.map
        except:
            return self.find_map()

    def rmse(self,test_data, predicted):
        low, high = self.bounds
        test_data[test_data < low] = low
        test_data[test_data > high] = high
        sqerror = abs(test_data - predicted) ** 2  # squared error array
        mse = sqerror.sum()/(test_data.shape[0]*test_data.shape[1])
        return np.sqrt(mse)

    def predict(self, U, V):
     #"""Estimate R from the given values of U and V."""
        R = np.dot(U, V.T)
        n, m = R.shape
        sample_R = np.random.poisson(R)
        # bound ratings
        low, high = self.bounds
        sample_R[sample_R < high] = low
        sample_R[sample_R >= high] = high
        return sample_R

    def eval_map(self, train, test):
        U = self.map["U"]
        V = self.map["V"]
        #VU = self.map["VU"]

        # Make predictions and calculate RMSE on train & test sets.
        predictions = self.predict(U, V )
        train_rmse = self.rmse(train, predictions)
        test_rmse = self.rmse(test, predictions)
        overfit = test_rmse - train_rmse

        # Print report.
        print("PMF MAP training RMSE: %.5f" % train_rmse)
        print("PMF MAP testing RMSE:  %.5f" % test_rmse)
        print("Train/test difference: %.5f" % overfit)

        return test_rmse

    def draw_samples(self, **kwargs):
        kwargs.setdefault("chains", 1)
        with self.model:
            self.trace = pm.sample(**kwargs)

In [3]:
import pickle

with open('data_all.pickle', 'rb') as handle:
    data = pickle.load(handle)



In [4]:

DIM = 10
pmf = PMF_vanilla(data, DIM)

INFO:root:building the PMF model
INFO:root:done building the PMF model


In [5]:
pmf.find_map()
pmf_map_rmse = pmf.eval_map(data, data)

INFO:root:finding PMF MAP using L-BFGS-B optimization...


INFO:root:found PMF MAP in 571 seconds


PMF MAP training RMSE: 0.44834
PMF MAP testing RMSE:  0.44834
Train/test difference: 0.00000


In [6]:

pmf.draw_samples(chains =1,
            draws=500,
            tune=100,)

Auto-assigning NUTS sampler...
INFO:pymc:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc:Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
INFO:pymc:Sequential sampling (1 chains in 1 job)
NUTS: [U, V]
INFO:pymc:NUTS: [U, V]


In [25]:

#although intercepts might not make that much sense in the poisson-gamma model
class PMF_intercepts:
    def __init__(self, train, dim):
        """Build the Probabilistic Matrix Factorization model using pymc3.



        """
        self.dim = dim
    
      
        self.data = train.copy()
        self.n, self.m = self.data.shape
        self.map = None
        self.bounds = (0,1)


        # Perform mean value imputation
    
        
        # Low precision reflects uncertainty; prevents overfitting.
        # Set to the mean variance across users and items.
        self.alpha_u = (np.mean(self.data, axis=1).mean())**2 / np.std(self.data, axis=1).mean()
        self.alpha_v = (np.mean(self.data, axis=0).mean())**2 / np.std(self.data, axis=0).mean()

        self.beta_u = (np.mean(self.data, axis=1).mean()) / np.std(self.data, axis=1).mean()
        self.beta_v = (np.mean(self.data, axis=0).mean()) / np.std(self.data, axis=0).mean()
       
        self.bias = self.data.mean()
        #specify model coords
        coords = {
        "drug": np.arange(self.n),
        "latent": np.arange(self.dim),
        "side_effect": np.arange(self.m),
            }
        # Specify the model.
        logging.info("building the PMF model")

        with pm.Model(coords=coords) as pmf:
            U = pm.Gamma(
                "U",
                alpha=self.alpha_u,
                beta=self.beta_u,
                dims=("drug", "latent"),
                #testval=np.random.randn(n, dim) * std,
            )
      
            V = pm.Gamma(
                "V",
                alpha=self.alpha_v,
                beta=self.beta_v,
                dims=("side_effect", "latent"),
                #testval=np.random.randn(m, dim) * std,
            )
    

            R = pm.Poisson(
                "R", mu=(U @ V.T), observed=self.data , dims = ("drug", "side_effect")
            )

        logging.info("done building the PMF model")
        self.model = pmf

    def __str__(self):
        return self.name
    
    def find_map(self):
    #"""Find mode of posterior using L-BFGS-B optimization."""
        tstart = time.time()
        with self.model:
            logging.info("finding PMF MAP using L-BFGS-B optimization...")
            self.map = pm.find_MAP(method="L-BFGS-B")

        elapsed = int(time.time() - tstart)
        logging.info("found PMF MAP in %d seconds" % elapsed)
        return self.map


    def map(self):
        try:
            return self.map
        except:
            return self.find_map()

    def rmse(self,test_data, predicted):
        low, high = self.bounds
        test_data[test_data < low] = low
        test_data[test_data > high] = high
        sqerror = abs(test_data - predicted) ** 2  # squared error array
        mse = sqerror.sum()/(test_data.shape[0]*test_data.shape[1])
        return np.sqrt(mse)

    def predict(self, U, V):
     #"""Estimate R from the given values of U and V."""
        R = np.dot(U, V.T)
        n, m = R.shape
        sample_R = np.random.poisson(R)
        # bound ratings
        low, high = self.bounds
        sample_R[sample_R < low] = low
        sample_R[sample_R > high] = high
        return sample_R

    def eval_map(self, train, test):
        U = self.map["U"]
        V = self.map["V"]
        #VU = self.map["VU"]

        # Make predictions and calculate RMSE on train & test sets.
        predictions = self.predict(U, V )
        train_rmse = self.rmse(train, predictions)
        test_rmse = self.rmse(test, predictions)
        overfit = test_rmse - train_rmse

        # Print report.
        print("PMF MAP training RMSE: %.5f" % train_rmse)
        print("PMF MAP testing RMSE:  %.5f" % test_rmse)
        print("Train/test difference: %.5f" % overfit)

        return test_rmse
    
    def draw_samples(self, **kwargs):
        kwargs.setdefault("chains", 1)
        with self.model:
            self.trace = pm.sample(**kwargs)

[[ 1  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 1  0  0 ...  1  8  0]
 ...
 [ 8  0  0 ... 10 12  0]
 [ 1  0  0 ...  4 25  0]
 [ 0  0  0 ...  0  0  0]]


[[ 1  0  0 ...  0  1  0]
 [ 0  0  0 ...  1  2  0]
 [ 0  0  0 ...  6  4  1]
 ...
 [11  1  0 ...  6 27  0]
 [ 7  0  0 ...  5 15  0]
 [ 0  0  0 ...  1  0  0]]
